# Transforming Elsner & Hoelzer to a new format for FracFocus integration

The full data spreadsheet from the paper was downloaded from the Environmental Science and Technology supporting information website.

This code uses the "categories full" sheet of that file.

**Manual changes to the file before processing:**
- Insert a new column (as 'B') named "Type-row"  that indicates a classification name (in "Chemical Component" field) for subsequent records.  Two levels.
- Edit line 55: change CAS field 
    **from:** 94266-47-4, 9426647468647-72-3
    ****to:**   94266-47-4, 68647-72-3
- Edit line 173: change CAS field
    **from:** 2426-08-65
    **to:**   2426-08-6
- Edit line 534: change CAS field
    **from:** " 9005-02-1"
    **to:**   '9005-02-1
- Edit line 702: Change CAS field
    **from:** “4719-04-4“
    **to:**   '4719-04-4
- Edit line 792:  Change CAS field
    **from:** 910644-97-2, '9003-05-8, *
    **to:**   910644-97-2, 9003-05-8, *
- Edit line 980:  Change CAS field
    **from:** "7783-06-4"
    **to:**   '7783-06-4
- Edit line 992:  Change CAS field
    **from:** 1113-47-9, '7632-04-4, 10486-00-7, 447-63-2, 10332-33-9
    **to:**   1113-47-9, 7632-04-4, 10486-00-7, 447-63-2, 10332-33-9
- Edit line 1038: Change CAS field
    **from:** 471-34-1 (calcite), 1317-65-3 (calcium carbonate)
    **to:**   471-34-1, 1317-65-3

CAS Numbers not found in SciFinder:
14464-46-4 **
6747-01-0 **
9004-34-6; **
61790-29-8 **
255043-08-04 **
27176-93-9 **
6132-04-31 **
9-89-2 **
1302-44-56 **
9018-45-9 **
9016-45-6 **
64742-97-5 **

In [3]:
import pandas as pd
import numpy as np
import csv

sources = "./sources/"
outdir = "./out/"

infn = sources+"ELSNER_with_modifications.xls"
outfn = outdir+"elsner_corrected_table.csv"

In [4]:
# show raw list of columns and their position
raw = pd.read_excel(infn,sheet_name=1,header=2)
raw.columns

Index(['Unnamed: 0', 'Type-row', 'FracFocus (SkyTruth)', 'SkyTruth 2011/12',
       'SkyTruth 2013', 'Waxman', 'FracFocus (own evaluation, spring 2014)',
       'FracFocus (EPA)', 'EPA entry (yes / no)', 'FracFocus (EPA) ',
       'FracFocus (Rogers et al.)', 'Chemical Abstract Service Number (CAS)',
       'Chemical Component', 'Substance class', 'Function',
       'Comment or Specification by FracFocus entries', 'Patent Number',
       'Log K ow estimated', 'Log K ow measured',
       'Water Solubility Estimate from log Kow (mg/L at 25°C)',
       'Henry's law constant (atm-m³/mol at 25°C)',
       'Henry's law constant (atm-m³/mol at 25°C).1',
       'Henry's law constant (atm-m³/mol at 25°C).2', ' t1/2 (d)',
       'degra- dation comments ', 'MCL/ HAI', 'MCL (mg L-1)',
       'RfD (mgkg-1 d-1)', 'OSF d (mg kg-1 d-1)', 'RfCd (mg m-3)',
       'log Koc est.', 'log Koc exp.', 'pKa', 'Kd (L*kg-1)', 'pred.≥ 10%'],
      dtype='object')

In [5]:
raw = pd.read_excel(infn,sheet_name=1,header=2,keep_default_na=False) 
raw = raw.filter(['Type-row','Chemical Abstract Service Number (CAS)',
                  'Chemical Component','Substance class','Function'], axis=1)
raw.rename(columns = {'Type-row':'typerow',
                      'Chemical Abstract Service Number (CAS)':'CAS',
                      'Chemical Component':'chemcomp',
                      'Substance class': 'sub_class',
                      'Function':'function'}, 
          inplace=True)
raw.head(20)


,typerow,CAS,chemcomp,sub_class,function
0,,,,,
1,,,,,
2,1,,Gases (Table 2 in Manuscript),,
3,,124-38-9,Carbon dioxide,gas - acid,fracking fluid
4,,7727-37-9,Nitrogen,gas - inert,fracking fluid
5,,,,,
6,1,,Non-functionalized Hydrocarbons (Table 2 in th...,,
7,2,,alkanes,,
8,,74-82-8,Methane,hydrocarbon - alkane,fracking fluid
9,,74-84-0,Ethane,hydrocarbon - alkane,fracking fluid


In [6]:
class New_set():
    def __init__(self):
        self.reclst = []
        self.Lvl1 = ''
        self.Lvl2 = ''
        
    def process_row(self,row):
        #print(row)
        if row.typerow==1:
            end = row.chemcomp.find('(')-1
            self.Lvl1 = row.chemcomp[:end]
            self.Lvl2 = ''
            return 'type 1' # that's all we do
        if row.typerow==2:
            end = row.chemcomp.find('(')
            if end==-1:
                end = len(row.chemcomp)
            self.Lvl2 = row.chemcomp[:end]
            return 'type 2'
        if row.CAS != '':
            l = row.CAS.split(',')
            if len(l)>0:
                for item in l:
                    s = item.replace('*','')
                    s = s.strip()
                    #print(s)
                    if s!='':
                        self.reclst.append((self.Lvl1,self.Lvl2,s,row.chemcomp.strip(),
                                            row.sub_class.strip(),
                                            row.function.strip()))
            return row.CAS
    def get_df(self):
        return pd.DataFrame(data=self.reclst,columns=['eh_Class_L1','eh_Class_L2','eh_CAS','eh_IngredientName',
                                                     'eh_subs_class','eh_function'])

In [7]:
ns = New_set()
test = raw.apply(lambda x: ns.process_row(x),axis=1)

In [8]:
ns.get_df().to_csv(outfn,index=False,quotechar='$',quoting=csv.QUOTE_ALL)